In [7]:
#검증 세트를 만들기

import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

#훈련, 테스트 세트 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

#훈련에서 검증 세트 분리
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state = 42)

print("sub_input.shape : ", sub_input.shape)   #sub_input.shape :  (4157, 3)
print("val_input.shape : ", val_input.shape)   #val_input.shape :  (1040, 3)

#의사결정나무로 만들기
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print("sub_input, sub_target 점수 : ", dt.score(sub_input, sub_target))
print("val_input, val_target 점수 : ", dt.score(val_input, val_target))

#sub_input, sub_target 점수 :  0.9971133028626413
#val_input, val_target 점수 :  0.864423076923077  => 과대적합인 것을 알 수 있다.

sub_input.shape :  (4157, 3)
val_input.shape :  (1040, 3)
sub_input, sub_target 점수 :  0.9971133028626413
val_input, val_target 점수 :  0.864423076923077


In [9]:
#교차 검증 만들기
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

#훈련, 테스트 세트 분리
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

#훈련에서 검증 세트 분리
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state = 42)

#교차검증
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print("scores : ", scores)

#scores :  {'fit_time': array([0.00500154, 0.00500131, 0.00500083, 0.00500131, 0.00500083]), 'score_time': array([0., 0., 0., 0., 0.]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
#=> fit time : 훈련시간, score time : 검증시간
#=> test_score : 각 키마다 5개의 숫자가 있다. 이것을 5개의 평균하여 최종점수를 얻어야 한다.cross_validate

#최종점수
import numpy as np
print(np.mean(scores['test_score']))  #0.855300214703487

###유의점, cross_validate() 는 훈련 세트를 섞어 폴드를 나누지 않는다. 그래서 교차 검증을 할 때 순련 세트를 섞으려면 분할기를 지정해야 한다.

scores :  {'fit_time': array([0.00500131, 0.00500131, 0.00500083, 0.00500131, 0.00500107]), 'score_time': array([0., 0., 0., 0., 0.]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [11]:
#10-폴드 교차 검증
from sklearn.model_selection import StratifiedKFold

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))   #0.8574181117533719

0.8574181117533719


In [ ]:
###248~262 그리드서치 추후 하기